In [1]:
if 'google.colab' in str(get_ipython()):
    import sys
    from google.colab import drive, output
    drive.mount('/content/drive')
    !pip install torchaudio
    !pip install wandb --upgrade
    output.clear()
    print("Running on colab")
    %load_ext autoreload
    %autoreload 1
    %cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
    root = "/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project"
else:
    print("Running local")
    root = "/home/mirp_ai/Documents/Daniel_Nieto/PhD/AutoEncoders_Ecoacoustics"
    root_path = "/media/mirp_ai/Seagate Desktop Drive/Jaguas_2018"


Running local


In [2]:
import os
import numpy as np

import datetime
from datetime import timedelta

import torch
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torchvision.utils import make_grid
from Jaguas_DataLoader_rainless import SoundscapeData

import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [233]:
class df_generator():
    def __init__(self, root_clusters, root_audios, root_recorders, root_save=None, n_clusters_index = 0):
        self.n_clusters_index = n_clusters_index
        self.cluster_names = os.listdir(f"{root_clusters}")
        self.torch_clusters = torch.load(f"{root_clusters}/{self.cluster_names[n_clusters_index]}")
        self.dataframe_clusters = pd.DataFrame(self.torch_clusters)
        self.dataframe_clusters = self.dataframe_clusters.transpose()
#         self.dataframe_audios = pd.read_csv(f"{root_audios}", index_col=0)
        self.dataframe_recorders = pd.read_csv(f"{root_recorders}", index_col = "Recorder")
        self.df_clusters_len = len(self.dataframe_clusters)
        self.df_recorders_len = len(self.dataframe_recorders)
        self.n_clusters = np.arange(len(self.torch_clusters))
        self.root_save = root_save

    def show_clusters(self):
        print(self.cluster_names)

    def init_clusters(self):
        for cluster in self.n_clusters:
            self.dataframe_recorders[f"Cluster {cluster}"] = 0
    
    def create_clusters(self, count_cluster=False):
        self.init_clusters()
        for cluster in self.n_clusters:
            for i in range(self.df_clusters_len):  
                if self.dataframe_clusters.iloc[i][cluster] != None:
                    self.dataframe_recorders.loc[self.dataframe_clusters.iloc[i][cluster].split("_")[0], f"Cluster {cluster}"] +=1
                else:
                    pass
        if count_cluster == True:
            self.count_cluster_data()
        else:
            pass
        
        return self.dataframe_recorders
    
    def count_cluster_data(self):
        self.dataframe_recorders["Total_Clustering_Audios"] = 0
        for recorder in range(self.df_recorders_len):
            clusters = [f"Cluster {i}" for i in range(0, len(self.n_clusters))]
            clusters.append("Total_Clustering_Audios")
            self.dataframe_recorders.iloc[recorder, -1] = self.dataframe_recorders[clusters].iloc[recorder].sum()

    def save(self):
        if self.root_save != None:
            os.makedirs(f"{self.root_save}/dataframes", exist_ok=True)
            self.dataframe_recorders.to_csv(f"{self.root_save}/dataframes/dataframe_{self.cluster_names[self.n_clusters_index]}.csv") 
        else:
            self.dataframe_recorders.to_csv(f"{self.cluster_names[sel.n_clusters_index]}")

In [234]:
root_audios = f"{root}/Jaguas/Complementary_Files/Audios_Jaguas/Audios_Jaguas.csv"
root_recorders = f"{root}//Jaguas/Complementary_Files/df_grabadoras.csv"
root_clusters = f"{root}/Jaguas/temporal/clusters"

In [235]:
dataframe = df_generator(root_clusters, root_audios, root_recorders, 
                         root_save = f"{root}/Jaguas/temporal/", n_clusters_index=0)
dataframe.create_clusters(count_cluster=True)

,Municipio,Vereda,Latitud,Longitud,Elevacion,Habitat,Total_grabaciones,Cluster 0,Cluster 1,Cluster 2,Cluster 3,Cluster 4,Total_Clustering_Audios
Recorder,,,,,,,,,,,,,
G03,Alejandria,El Respaldo,6.4137,-75.0366,1322,Forest,565,906,113,502,321,438,2280
G04,San Roque,Playa Rica,6.4286,-75.0281,1310,Forest,418,0,0,23,1461,146,1630
G06,Alejandria,El Respaldo,6.4088,-75.0453,1302,Forest,887,583,0,237,2586,139,3545
G07,San Roque,El Nucito,6.4287,-75.0368,1296,Non- forest,621,1231,1,838,327,123,2520
G08,Alejandria,El Respaldo,6.4061,-75.0534,1314,Non- forest,648,875,89,474,493,489,2420
G09,Santo Domingo,Nusito-Dantas,6.4204,-75.0474,1346,Forest,908,7,0,37,3422,104,3570
G13,Alejandria,El Respaldo,6.3968,-75.0186,1349,Forest,636,1038,4,1081,87,485,2695
G15,San Roque,Playa Rica,6.4045,-75.0143,1325,Forest,874,2969,0,252,236,113,3570
G17,Alejandria,El Respaldo,6.3978,-75.0444,1389,Non- forest,870,2138,109,453,20,930,3650


In [236]:
dataframe.save()

In [ ]:
os.chdir(root_path) #Ubicarnos en el directorio
print("Ubicados en :",dir)
!ls

In [ ]:
model = "PositionalAE_hour"
identifier = "No_rain_Audios_Jaguas"
day = 27
hour = 19
month = 6
folder = "AE_No_rain"
date_format = f"day_{day}_hour_{hour}"
path = torch.load(f'{root}/Jaguas/temporal/Features/{folder}/test_path_samples_{date_format}_{identifier}.pth', map_location=torch.device('cpu'))


In [ ]:
path[0]